## Импорт данных RLMS

In [4]:
import pyreadstat
#import pandas as pd

In [11]:
#df1_estim = pd.read_csv("ha_ts.csv",index_col=0)
df, meta = pyreadstat.read_sav("dta.sav")
#df
#df1_estim

,id,year,log_income,hours
32419,1.0,1997.0,5.683580,30.0
43094,1.0,1998.0,6.606650,30.0
54070,1.0,1999.0,7.517521,35.0
66191,1.0,2000.0,7.824046,22.0
78714,1.0,2001.0,9.277999,22.0
...,...,...,...,...
371871,59393.0,2017.0,10.596635,48.0
371872,59394.0,2017.0,11.141862,35.0
371875,59398.0,2017.0,10.126631,40.0
371876,59400.0,2017.0,9.392662,40.0


## Обработка данных

Анализируется зависимость трудового дохода в последние 30 дней от средней длины рабочей недели

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = df.loc[:,["IDIND", "ID_W", "J6.2", "J10"]]
df1.rename(columns={"IDIND": "id", "ID_W":"year", "J6.2":"hours", "J10":"income"}, inplace=True)

In [ ]:
year_new = df1.loc[:,"year"] + 1989
df1.loc[:,"year"] = year_new

In [ ]:
log_income = np.log(df1.loc[:,"income"])
df1.loc[:,"log_income"] = log_income
#print(df1)

In [ ]:
df1_clean = df1.replace([99999997, 99999998, 99999999], np.NaN).dropna()
#print(df1_clean)

In [ ]:
df1_sorted = df1_clean.sort_values(by=["id", "year"])
#print(df1_sorted)

## Визуализация данных за 2010 год

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df1_plot = df1_sorted[(df1_sorted["year"]==2010)]
#print(df1_plot)

In [ ]:
plt.scatter(x="hours", y="log_income", data=df1_plot)
plt.xlabel("Average working week, hours")
plt.ylabel("Log of income in last 30 days, roubles")
plt.title("Effect of longer working hours on income in 2010\n")
plt.show()

## Подготовка к построению моделей

In [ ]:
df1_estim = df1_sorted.loc[:,["id","year","log_income","hours"]]
df1_estim.to_csv("ha_ts.csv",index=True)
#print(df1_estim)

In [13]:
import statsmodels.api as sm

In [14]:
df1_mod = df1_estim.set_index(["id", "year"])
dep = df1_mod["log_income"]
exog = sm.add_constant(df1_mod[["hours"]])

### Оценка сквозной модели

In [15]:
from linearmodels import PooledOLS

In [17]:
mod1 = PooledOLS(dep, exog)
res1 = mod1.fit(cov_type="robust")
#print(res1)

### Оценка модели с фиксированными эффектами

In [18]:
from linearmodels import PanelOLS

Установим параметр entity_effects=True для проверки гипотезы о равенстве индивидуальных эффектов и сравнения модели с фиксированными эффектами со сквозной моделью

In [19]:
mod2 = PanelOLS(dep, exog, entity_effects=True)
res2 = mod2.fit(cov_type='robust')
#print(res2)

Гипотеза о равенстве индивидуальных эффектов отвергается при любом разумном уровне значимости, поэтому лучше использовать модель с фиксированными эффектами

### Оценка модели со случайными эффектами

In [20]:
from linearmodels import RandomEffects

In [21]:
mod3 = RandomEffects(df1_mod.log_income, exog)
res3 = mod3.fit(cov_type="robust")
#print(res3)

### Сравнение оценок разных моделей

In [22]:
from linearmodels.panel import compare
print(compare({'Pooled':res1,'FE':res2,'RE':res3}))

                            Model Comparison                           
                                Pooled             FE                RE
-----------------------------------------------------------------------
Dep. Variable               log_income     log_income        log_income
Estimator                    PooledOLS       PanelOLS     RandomEffects
No. Observations                130474         130474            130474
Cov. Est.                       Robust         Robust            Robust
R-squared                       0.0147         0.0038            0.6349
R-Squared (Within)             -0.0007         0.0038            0.0035
R-Squared (Between)             0.0178         0.0110            0.0149
R-Squared (Overall)             0.0147         0.0107            0.0119
F-statistic                     1941.8         390.68         2.268e+05
P-value (F-stat)                0.0000         0.0000            0.0000
=====================     ============   ============   ========

### Тест Хаусмана для выбора между FE и RE

In [23]:
from scipy.stats import chi2

Оценки коэффициентов моделей RE и FE соответственно

In [24]:
q_re = res3.params["hours"]
q_fe = res2.params["hours"]

Расчет статистики Хаусмана

In [25]:
v_re = res3.std_errors["hours"] ** 2
v_fe = res2.std_errors["hours"] ** 2

In [26]:
q = q_fe - q_re
v = v_fe - v_re

In [27]:
m = q * v ** (-1) * q
#print(m)

Расчет критического значения

In [28]:
q = 0.95
doff = res3.df_model
m_cr = chi2.ppf(q, doff)
#print(m_cr)

H0: ковариация случайного эффекта с регрессорами равна нуля
H1: -//- не равна нулю

In [29]:
if m > m_cr:
    print("H_0 отвергается")
else:
    print("H_0 не отвергается")

H_0 отвергается


Следовательно, лучше использовать модель с фиксированным эффектом

Ответ на главный вопрос: 42

Ответ на другой главный вопрос: доход за последние 30 дней и средняя длинна рабочей недели взаимосвязаны при любом разумном уровне значимости по результатам тестов. 